# COMP 551 AML Assignment 2
## Ran Cheng, 260768706
This project is for assignment 2 of applied machine learning course.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# read classifier data, mean
ds1_m0_data = pd.read_csv('./Data/DS1_m_0.txt', sep=",", header=None)
ds1_m0_data = ds1_m0_data.dropna(axis=1, how='all')
ds1_m0 = np.squeeze(np.asarray(ds1_m0_data))
ds1_m1_data = pd.read_csv('./Data/DS1_m_1.txt', sep=",", header=None)
ds1_m1_data = ds1_m1_data.dropna(axis=1, how='all')
ds1_m1 = np.squeeze(np.asarray(ds1_m1_data))
print(ds1_m0, ds1_m1)

[1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3
 1.3 1.3] [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [3]:
# read covarience matrrix
ds1_cov_data = pd.read_csv('./Data/DS1_Cov.txt', sep=",", header=None)
ds1_cov_data = ds1_cov_data.dropna(axis=1, how='all')
ds1_cov = np.squeeze(np.asarray(ds1_cov_data))
ds1_cov.shape

(20, 20)

In [4]:
# get the data for classifier, 0
out_0 = np.random.multivariate_normal(ds1_m0, ds1_cov, size=2000)
label_0 = np.zeros((2000,1))
labeled_ds1_0 = np.hstack((out_0, label_0))
# get the data for classifier, 1
out_1 = np.random.multivariate_normal(ds1_m1, ds1_cov, size=2000)
label_1 = np.ones((2000,1))
labeled_ds1_1 = np.hstack((out_1, label_1))
print(labeled_ds1_0[0], labeled_ds1_1[1])

[ 4.14491196  1.53017126  1.71401632  3.54927498  2.07323979  1.87764833
  2.32451662  2.73364386  2.39486848  2.91365673  2.76285416  3.39611582
  3.96712059  1.43105483 -0.24021163  0.66680738  2.3041561   3.0002512
  1.31557552  2.91083169  0.        ] [-3.15529136 -2.3136306  -5.20673933 -2.77839985 -2.99008919 -3.48616892
 -2.44907064 -2.55133267 -1.0388036  -1.02870932 -0.41038205 -2.75029332
 -3.72827684 -1.090578   -2.98097217 -3.20934366 -3.36349414 -3.28055173
 -1.72360436 -0.6979119   1.        ]


In [5]:
# concaternate negative and positive data
merged_ds1 = np.vstack((labeled_ds1_0,labeled_ds1_1))
# shuffle the negative and positive data
np.random.shuffle(merged_ds1)
# get the data split into two parts randomly: training and testing data
train, test = train_test_split(merged_ds1, test_size=0.3)
n = train.shape[0]
n_test = test.shape[0]

In [6]:
mu1 = np.mean(train[train[0:,-1] == 1][0:,0:-1], axis=0) #mu1 hat
mu0 = np.mean(train[train[0:,-1] == 0][0:,0:-1], axis=0) #mu1 hat
mu0.shape

(20,)

In [7]:
n_0 = len(train[train[0:,-1] == 0])
n_1 = len(train[train[0:,-1] == 1])
p_y0 = n_0/n
p_y1 = n_1/n
print('n_0: % d,' % n_0, ' n_1: % d' % n_1)

n_0:  1424,  n_1:  1376


In [8]:
s1 = train[train[0:,-1] == 1][0:, 0:-1] - mu1
sig1 =  np.dot(s1.transpose(), s1)
s0 = train[train[0:,-1] == 1][0:, 0:-1] - mu0
sig0 =  np.dot(s0.transpose(), s0)
sigma = (sig0 + sig1)/n
invsig = np.linalg.inv(sigma)

In [16]:
# estimate w0 and w1
w_0 = np.log(p_y0) - np.log(p_y1) - (1/2)*(np.dot(np.dot(mu0.transpose(), invsig), mu0)) + (1/2)*(np.dot(np.dot(mu1.transpose(), invsig), mu1))
w_1 = np.dot(invsig, (mu0 - mu1))
print('w0: %0.4f' % w_0, 'w1: ', w_1)

w0: 4.0919 w1:  [ 2.14051524 -1.27070647 -0.87450193 -0.45329787 -1.45965003 -0.6318571
  2.55355408 -3.59993189 -4.35064918  1.35902226 -1.97149091 -1.83832722
  2.34055644  1.97955225 -0.81823733  1.93790251  4.36248385 -1.00938253
 -0.1011214  -0.75919109]


In [10]:
tmp = np.dot(test[0:,0:-1], w_1)
result = w_0 + tmp
result[result >= 0] = 0
result[result < 0] = 1
error = sum(result != test[:,-1])
print("The error rate of the estimation on the testing set is {0}/{1} or : {2}%"\
          .format(error, n_test, 100*error/n_test))

The error rate of the estimation on the testing set is 56/1200 or : 4.666666666666667%


In [14]:
# fit metric
from sklearn.metrics import classification_report
y_true = test[:,-1]
y_pred = result
print(classification_report(y_true, y_pred, target_names=['class 0', 'class 1']))

             precision    recall  f1-score   support

    class 0       0.95      0.95      0.95       576
    class 1       0.95      0.96      0.96       624

avg / total       0.95      0.95      0.95      1200

